In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=74d15e29c16eb81f168c571672aab95ac4c65b6bec315d4a54944e99dbfd5ca8
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [4]:
# Montage drive (optionnel)
from google.colab import drive
drive.mount('/My_drive/')

Mounted at /My_drive/


In [5]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.util import MLUtils
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col,lower
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.functions import regexp_extract, when
from pyspark.ml.feature import Word2Vec



sc

<SparkContext master=local[*] appName=pyspark-shell>

In [6]:
#Charger un csv (modifier le path)
# path="/My_drive/My Drive/Cours prof Big data/2022 GIS5/Projet/offensive_comment_small.csv"
path="/My_drive/My Drive/big_data_tp/offensive_comment_small.csv"
df = spark.read.csv(path,header=True, sep=";")
df=df.dropna()
print(df.count())
df.show()


9998
+------------+--------------------+-----+------------+-------+------+------+-------------+
|          id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|
+------------+--------------------+-----+------------+-------+------+------+-------------+
| 86236166566|How can Ceha use ...|    0|           0|      0|     0|     0|            0|
| 67516955649|Blah... Childlove...|    0|           0|      0|     0|     0|            0|
| 22144619496|WTH DUDE Deleting...|    0|           0|      0|     0|     1|            0|
| 10428262239|"'""I''ve added a...|    0|           0|      0|     0|     0|            0|
| 82779776033|Violation Your re...|    0|           0|      0|     0|     0|            0|
| 72769334443|take care  16:55,...|    0|           0|      0|     0|     0|            0|
| 80250530594|'The second examp...|    0|           0|      0|     0|     0|            0|
| 40748094103|"'"" Bye I''m not...|    1|           0|      1|     0|     1|         

# Nettoyage et relabélisation

## Relabélisation

Les phrases contenant l mot "fuck" sont censé ête toxique, je classerai donc comme toxique les observations qui contiennent ce mot.

In [7]:
df_fuck = df.withColumn("contient_fuck", when(regexp_extract("comment_text", "fuck", 0) != "", 1).otherwise(0))
df_fuck = df_fuck.withColumn("comment_text", lower(df_fuck["comment_text"]))
#df_fuck.select(df_fuck.comment_text,df_fuck.contient_fuck).show(truncate=False)
df_fuck = df_fuck.withColumn("poid", df_fuck.contient_fuck*4+df_fuck.toxic) #si la chaine de caractères comment_text contient le mot fuck on donne un poids 5 fois supérieures à cette observation
df_fuck.show(truncate=False)

+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Nettoyer la chaine de caractère

Grâce à la fonction clean_text et à la fonction clean_text_2 on en enlève les mots qui n'ont pas d'utilité dans l'étude que nous allons réaliser : les virgules , déterminants etc ... 

In [8]:
# Dans un premier temps, faire une fonction qui fonctionne sur un exemple simple

import spacy

nlp = spacy.load("en_core_web_sm", disable=["ner"])

def clean_text(text):
  doc = nlp(text)
  tab_result = []
  # Ne garder que les LEMME, enlever les STOPWORDS et les mots de moins de 2 charactères
  for token in doc:
      if ( (not token.is_stop) and len(token.text)>2 ) :
        tab_result.append(token.lemma_)
  return tab_result

# modifier la phrase 
clean_text("This IS a test, we are currently in a big data lesson and we try spacy")

['test', 'currently', 'big', 'datum', 'lesson', 'try', 'spacy']

In [9]:

#colonne créé: text_cleaned
#fonction utilisée pour la créer 'clean_text'
#type de retours  T.ArrayType(T.StringType())
#colonne sur laquelle appliquer la fonction 'comment_text'
 
df_clean_text= df.withColumn(
    'text_cleaned', 
    F.udf(
        clean_text
        ,T.ArrayType(T.StringType())
    )("comment_text")
)

In [10]:
#pour le tableau avec les poids

df_clean_text_fuck= df_fuck.withColumn(
    'text_cleaned', 
    F.udf(
        clean_text
        ,T.ArrayType(T.StringType())
    )("comment_text")
)

In [ ]:
# cellule isolée car long temps de calcul 
df_clean_text.write.parquet("/My_drive/My Drive/BigData/Projet/first_clean.pqt",mode="overwrite")
df_clean_text_fuck.write.parquet("/My_drive/My Drive/BigData/Projet/second_clean.pqt",mode="overwrite")

In [11]:
df_clean_text = spark.read.parquet("/My_drive/My Drive/BigData/Projet/first_clean.pqt")
df_clean_text_fuck=spark.read.parquet("/My_drive/My Drive/BigData/Projet/second_clean.pqt")


On cmet les colonnes dan le bon type

In [12]:
# cast
#df = df_vectorized
df_clean_text= df_clean_text.withColumn('toxic', df_clean_text["toxic"].cast("int"))
df_clean_text= df_clean_text.withColumn('severe_toxic', df_clean_text["severe_toxic"].cast("int"))
df_clean_text= df_clean_text.withColumn('obscene',df_clean_text["obscene"].cast("int"))
df_clean_text= df_clean_text.withColumn('threat', df_clean_text["threat"].cast("int"))
df_clean_text= df_clean_text.withColumn('insult', df_clean_text["insult"].cast("int"))
df_clean_text=df_clean_text.withColumn('identity_hate', df_clean_text["identity_hate"].cast("int"))
df_clean_text.printSchema()
df_clean_text=df_clean_text.dropna()
df_clean_text.count()

root
 |-- id: string (nullable = true)
 |-- comment_text: string (nullable = true)
 |-- toxic: integer (nullable = true)
 |-- severe_toxic: integer (nullable = true)
 |-- obscene: integer (nullable = true)
 |-- threat: integer (nullable = true)
 |-- insult: integer (nullable = true)
 |-- identity_hate: integer (nullable = true)
 |-- text_cleaned: array (nullable = true)
 |    |-- element: string (containsNull = true)



9998

In [13]:
df_clean_text_fuck= df_clean_text_fuck.withColumn('toxic', df_clean_text_fuck["toxic"].cast("int"))
df_clean_text_fuck= df_clean_text_fuck.withColumn('contient_fuck', df_clean_text_fuck["contient_fuck"].cast("int"))
df_clean_text_fuck= df_clean_text_fuck.withColumn('poid', df_clean_text_fuck["poid"].cast("int"))
df_clean_text_fuck= df_clean_text_fuck.withColumn('severe_toxic', df_clean_text_fuck["severe_toxic"].cast("int"))
df_clean_text_fuck= df_clean_text_fuck.withColumn('obscene',df_clean_text_fuck["obscene"].cast("int"))
df_clean_text_fuck= df_clean_text_fuck.withColumn('threat', df_clean_text_fuck["threat"].cast("int"))
df_clean_text_fuck= df_clean_text_fuck.withColumn('insult', df_clean_text_fuck["insult"].cast("int"))
df_clean_text_fuck=df_clean_text_fuck.withColumn('identity_hate', df_clean_text_fuck["identity_hate"].cast("int"))
df_clean_text_fuck.printSchema()
df_clean_text_fuck=df_clean_text_fuck.dropna()
df_clean_text_fuck.count()

root
 |-- id: string (nullable = true)
 |-- comment_text: string (nullable = true)
 |-- toxic: integer (nullable = true)
 |-- severe_toxic: integer (nullable = true)
 |-- obscene: integer (nullable = true)
 |-- threat: integer (nullable = true)
 |-- insult: integer (nullable = true)
 |-- identity_hate: integer (nullable = true)
 |-- contient_fuck: integer (nullable = true)
 |-- poid: integer (nullable = true)
 |-- text_cleaned: array (nullable = true)
 |    |-- element: string (containsNull = true)



9998

# Entrainement de modèles

## CountVectorizer

J'ai essayé plusieurs approches. Un countVectorizer avec un vocabulaire de 200 mots puis avec un de 300. 


In [14]:
from pyspark.ml.feature import CountVectorizer
# fit a CountVectorizerModel from the corpus.
cv = CountVectorizer(inputCol="text_cleaned", outputCol="Vectorized", vocabSize=200, minDF=2.0)
cv_300= CountVectorizer(inputCol="text_cleaned", outputCol="Vectorized", vocabSize=300, minDF=2.0)
#changer vocabsize 
model = cv.fit(df_clean_text)
model_fuck=cv_300.fit(df_clean_text_fuck)
df_vectorized = model.transform(df_clean_text)
df_vectorized_fuck=model_fuck.transform(df_clean_text_fuck)
# result.show(truncate=False)

#df_vectorized.select(["id","comment_text","text_cleaned","Vectorized"]).show(truncate=False)

#print(len(model.vocabulary))


# affiche : taille du dictionnaire, index des mots, nb de repetition

Un randomn forest avant l'augmentation de la taille

Dans un premier temps j'ai entrainé mon modèle avec un randomn forest par grid search en l'évualant avec le critère AUC et puis avec un F1 pour prendre en compte la répartition inégale de la classe toxique. Après en avoir discuté avec l'enseignant la il se trouavait que la méthode F1 nétait pas la plus adaptée 

In [15]:
#avant le nettoyage
#grid search randomn forest
#penser à rajouter plus d'arbres quand on va augmenter le nombre de mots dans le vocabulaires
(trainingData, testData) = df_vectorized.randomSplit([0.7, 0.3], seed=42)
featureIndexer = VectorIndexer(inputCol="Vectorized", outputCol="indexedFeatures", maxCategories=4).fit(df_vectorized)
rf = RandomForestClassifier(labelCol="toxic", featuresCol="indexedFeatures")
pipeline = Pipeline(stages=[featureIndexer, rf])

paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20,25]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .build()

evaluator=MulticlassClassificationEvaluator(labelCol="toxic",predictionCol="prediction",metricName="fMeasureByLabel")
cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=3)
cvModel = cv.fit(trainingData)
predictions = cvModel.transform(testData)
fmeasure = evaluator.evaluate(predictions)
print("fmeasurebyLabel = %g" % fmeasure)







fmeasurebyLabel = 0.917684


In [16]:

tp = predictions[(predictions.toxic == 1) & (predictions.prediction == 1)].count()
fn=  predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].count()
fp=  predictions[(predictions.toxic == 0) & (predictions.prediction == 1)].count()
print("True positives = {}".format(tp))
print("False negatives = {} \n".format(fn))
print("False positives = {} \n".format(fp))

True positives = 148
False negatives = 414 

False positives = 4 



In [17]:
 predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].show(truncate=False)

+------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+------------+-------+------+------+-------------+------------------

Analyse des faux negatifs

On remarque le modèles a du mal à prédire ces phrases comme toxiques alors qu'elles le sont. On remarque que le mot "fuck" apparait beaucoup dans les phrases labellisé par toxique. C'est pourquoi j'ai envisagé de donné un poid 4 foid plus important au phrase contenant les lettres "fuck" à la suite.

In [18]:
predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].show(truncate=False)

+------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+------------+-------+------+------+-------------+------------------

Grid Search après le nouveau nettoyage


In [19]:
#grid search randomn forest
#penser à rajouter plus d'arbres quand on va augmenter le nombre de mots dans le vocabulaires


(trainingData, testData) = df_vectorized_fuck.randomSplit([0.7, 0.3], seed=42)
featureIndexer = VectorIndexer(inputCol="Vectorized", outputCol="indexedFeatures", maxCategories=4).fit(df_vectorized_fuck)
rf = RandomForestClassifier(labelCol="toxic", featuresCol="indexedFeatures",weightCol="poid")
pipeline = Pipeline(stages=[featureIndexer, rf])

paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20,25]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .build()

evaluator=MulticlassClassificationEvaluator(labelCol="toxic",predictionCol="prediction",metricName="fMeasureByLabel")
cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=3)
cvModel = cv.fit(trainingData)
predictions = cvModel.transform(testData)

AUC = evaluator.evaluate(predictions)

print("fmeasure = %g" % AUC)
#print("F1= %g" %F1 )
#bestParams = cvModel.bestModel.stages[-1].extractParamMap()
#print("Les meilleurs paramètres de la méthode avec l'AUC sont : \n {}".format(bestParams))






fmeasure = 0
Avec l'AUC 



In [20]:
tp = predictions[(predictions.toxic == 1) & (predictions.prediction == 1)].count()
fn=  predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].count()
fp=  predictions[(predictions.toxic == 0) & (predictions.prediction == 1)].count()

print("True positives = {}".format(tp))
print("False negatives = {} \n".format(fn))
print("False positifs = {} \n".format(fp))

True positives = 562
False negatives = 0 

False positifs = 2334 



On remarque une augmentation du nombre de vrai positifs et une diminution de faux négatifs. Le modèle semble meilleur.

- La métrique F1 est une moyenne harmonique de la précision et du rappel, qui sont deux mesures de la performance d'un modèle de classification. Elle est souvent utilisée lorsque les classes sont équilibrées et que l'on s'intéresse à la performance globale du modèle.

- La métrique F-measure by label est une variante de la métrique F1 qui calcule la précision, le rappel et la F1 pour chaque classe individuelle, puis calcule une moyenne pondérée des F1 scores pour toutes les classes. Elle est souvent utilisée lorsque les classes ne sont pas équilibrées et que l'on s'intéresse à la performance de chaque classe individuellement.


J'avais d'abord choisis la métrique F1 parce que je pensais qu'lle prenait en compte le déséquilibre.

In [43]:

(trainingData, testData) = df_vectorized.randomSplit([0.7, 0.3], seed=42)

featureIndexer = VectorIndexer(inputCol="Vectorized", outputCol="indexedFeatures", maxCategories=4).fit(df_vectorized)
rf = RandomForestClassifier(labelCol="toxic", featuresCol="indexedFeatures")
pipeline = Pipeline(stages=[featureIndexer, rf])

paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20,25,30]) \
    .addGrid(rf.maxDepth, [5, 10,15]) \
    .build()


evaluator= MulticlassClassificationEvaluator(labelCol="toxic", predictionCol="prediction", metricName="f1")
evaluator2=MulticlassClassificationEvaluator(labelCol="toxic",predictionCol="prediction",metricName="fMeasureByLabel")

cv= CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=3)
cv2= CrossValidator(estimator=pipeline, evaluator=evaluator2, estimatorParamMaps=paramGrid, numFolds=3)

cvModel= cv.fit(trainingData)
cvModel2=cv2.fit(trainingData)
predictions= cvModel.transform(testData)
predictions2=cvModel2.transform(testData)

f1=evaluator.evaluate(predictions)
fmeasure=evaluator2.evaluate(predictions2)





In [45]:
f= evaluator.evaluate(predictions)
print("fmeasure = %g" % f)
tp2= predictions2[(predictions2.toxic == 1) & (predictions2.prediction == 1)].count()
fn2 = predictions2[(predictions2.toxic == 1) & (predictions2.prediction == 0)].count()
fp2 = predictions2[(predictions2.toxic == 0) & (predictions2.prediction == 1)].count()

print("True positives = {}".format(tp2))
print("False positives = {}".format(fn2))
print("False positives = {}".format(fp2))



fmeasure = 0.837586
True positives = 181
False positives = 381
False positives = 7


True positives = 181
False positives = 381
False positives = 7


Les faux négatifs

In [24]:
#vérifions les résultats pour voir s'assurer des perfomrances du modèle
predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].show(truncate=False)


+------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+------------+-------+------+------+-------------+------------------

Arbre de décision


In [25]:

#arbre de décision avec une validation croisée avec des hyperparamètres de validation croisée choisis à la main

(trainingData, testData) = df_vectorized.randomSplit([0.7, 0.3], seed=42)
featureIndexer=VectorIndexer(inputCol="Vectorized", outputCol="indexedFeatures", maxCategories=4).fit(df_vectorized)
dt = DecisionTreeClassifier(featuresCol="indexedFeatures", labelCol="toxic")
pipeline = Pipeline(stages=[featureIndexer, dt])


# definition des parametres de la validation croisée

paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [5, 10, 15]) \
    .addGrid(dt.minInstancesPerNode, [1, 5, 10]) \
    .build()


evaluator=MulticlassClassificationEvaluator(labelCol="toxic",predictionCol="prediction",metricName="fMeasureByLabel")
#instanciation de l'objet de validation croisée

cv=CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)

#entrainement du modèle

cvModel=cv.fit(trainingData)
predictions = cvModel.transform(testData)
auc = evaluator.evaluate(predictions)


In [78]:
print("fmeasurebylabel = %g" % auc)
tp = predictions[(predictions.toxic == 1) & (predictions.prediction == 1)].count()
fn = predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].count()
fp=  predictions[(predictions.toxic == 0) & (predictions.prediction == 1)].count()
print("True positives = {}".format(tp)  )
print("False negatives = {}".format(fn))
print("False positives = {}".format(fp))


fmeasurebylabel = 0.9248
True positives = 208
False negatives = 354
False positives = 22


ici on va essayer de rajouter les nouveaux poids dans le MulticlassClassificationEvaluator

In [48]:

#arbre de décision avec une validation croisée avec des hyperparamètres de validation croisée choisis à la main

(trainingData, testData) = df_vectorized_fuck.randomSplit([0.7, 0.3], seed=42)
featureIndexer=VectorIndexer(inputCol="Vectorized", outputCol="indexedFeatures", maxCategories=4).fit(df_vectorized_fuck)
dt = DecisionTreeClassifier(featuresCol="indexedFeatures", labelCol="toxic",weightCol="poid")
pipeline = Pipeline(stages=[featureIndexer, dt])

# definition des parametres de la validation croisée

paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [5, 10, 15]) \
    .addGrid(dt.minInstancesPerNode, [1, 5, 10]) \
    .build()


evaluator=MulticlassClassificationEvaluator(labelCol="toxic",predictionCol="prediction",metricName="fMeasureByLabel")
#instanciation de l'objet de validation croisée

cv=CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)

#entrainement du modèle

cvModel=cv.fit(trainingData)

predictions = cvModel.transform(testData)

auc = evaluator.evaluate(predictions)

In [49]:
print("fmeasurebylabel = %g" % auc)
tp = predictions[(predictions.toxic == 1) & (predictions.prediction == 1)].count()
fn = predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].count()
fp=  predictions[(predictions.toxic == 0) & (predictions.prediction == 1)].count()
print("True positives = {}".format(tp)  )
print("False negatives = {}".format(fn))
print("False positives = {}".format(fp))



fmeasurebylabel = 0
True positives = 562
False negatives = 0
False positives = 2334


Cette opération n'a pas permi d'augmenter le nombre de vrai positifs ni de diminuer les faux négatifs bien au contraire donc c'est un échec avec les arbres de décisions peut être qu'avec un autre modèle les résultats seraient plus concluants.

In [33]:
print("fmeasurebylabel = %g" % auc)
tp = predictions[(predictions.toxic == 1) & (predictions.prediction == 1)].count()
fn = predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].count()
fp=  predictions[(predictions.toxic == 0) & (predictions.prediction == 1)].count()
print("True positives = {}".format(tp)  )
print("False negatives = {}".format(fn))
print("False positives = {}".format(fp))

fmeasurebylabel = 0.9248
True positives = 208
False negatives = 354


### essai avec la métrique f1, mais pas recommandé car les classes sont déséquilibrées

In [ ]:

#arbre de décisionavec un grid search avec la métrique f1

(trainingData, testData) = df_vectorized.randomSplit([0.7, 0.3], seed=42)
featureIndexer=VectorIndexer(inputCol="Vectorized", outputCol="indexedFeatures", maxCategories=4).fit(df_vectorized)
dt = DecisionTreeClassifier(featuresCol="indexedFeatures", labelCol="toxic")
pipeline = Pipeline(stages=[featureIndexer, dt])


# definition des parametres de la validation croisée

paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [5, 10, 15]) \
    .addGrid(dt.minInstancesPerNode, [1, 5, 10]) \
    .build()


evaluator = MulticlassClassificationEvaluator(labelCol='toxic', predictionCol='prediction', metricName='f1')

#instanciation de l'objet de validation croisée

cv=CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)

#entrainement du modèle

cvModel=cv.fit(trainingData)

predictions = cvModel.transform(testData)

f1 = evaluator.evaluate(predictions)
print("f1 = %g" % f1)

#bestParams = cvModel.bestModel.stages[-1].extractParamMap()

#print("Les meilleurs paramètres sont : \n {}".format(bestParams))

#meilleure performance avec les arbres de décisions 

AUC = 0.847889
True positives = 214
False positives = 30


In [31]:
tp = predictions[(predictions.toxic == 1) & (predictions.prediction == 1)].count()
fn = predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].count()

print("True positives = {}".format(tp)  )
print("False positives = {}".format(fn))


True positives = 210
False positives = 352


## Diminution de la taille du vocabulaire

On va diminuer la taille du vocabulaire considéré pour voir comment cela influe sur la précision de nos prédictions

In [46]:
#countvectorizer part II

cv = CountVectorizer(inputCol="text_cleaned", outputCol="Vectorized", vocabSize=150, minDF=2.0)
model = cv.fit(df_clean_text)
df_vectorized2 = model.transform(df_clean_text)
# result.show(truncate=False)

#df_vectorized.select(["id","comment_text","text_cleaned","Vectorized"]).show(truncate=False)


On remarque une dimunition des performances de notre modèle avec un vocabulaire de 150 mots avec un AUC de 86.39 % contre 86,94% toutefois on remarque une diminutin des faux positifs et vrais positifs . Le modèle semble moins bon.

In [45]:
#arbre de décisions

#arbre de décision avec une validation croisée avec des hyperparamètres de validation croisée choisis à la main

(trainingData, testData) = df_vectorized2([0.7, 0.3], seed=42)
featureIndexer=VectorIndexer(inputCol="Vectorized", outputCol="indexedFeatures", maxCategories=4).fit(df_vectorized2)
dt = DecisionTreeClassifier(featuresCol="indexedFeatures", labelCol="toxic")
pipeline = Pipeline(stages=[featureIndexer, dt])

# definition des parametres de la validation croisée

paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [5, 10, 15]) \
    .addGrid(dt.minInstancesPerNode, [1, 5, 10]) \
    .build()


evaluator = MulticlassClassificationEvaluator(labelCol='toxic', predictionCol='prediction', metricName='fMeasureByLabel')

#instanciation de l'objet de validation croisée

cv=CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)

#entrainement du modèle

cvModel=cv.fit(trainingData)

predictions = cvModel.transform(testData)

accuracy = evaluator.evaluate(predictions)
print("fmbylabel = %g" % accuracy)

bestParams = cvModel.bestModel.stages[-1].extractParamMap()




f1 = 0.933145


In [59]:
print("fmeasurebylabel = %g" % auc)
tp = predictions[(predictions.toxic == 1) & (predictions.prediction == 1)].count()
fn = predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].count()
fp=  predictions[(predictions.toxic == 0) & (predictions.prediction == 1)].count()
print("True positives = {}".format(tp)  )
print("False negatives = {}".format(fn))
print("False positives = {}".format(fp))

True positives = 2
False negatives = 560


#  IDF

In [60]:
#randomn forest

#L'argument numFeatures dans la fonction HashingTF spécifie 
#le nombre de fonction de hachage à utiliser pour la transformation des mots en vecteurs
hashingTF = HashingTF(inputCol="text_cleaned", outputCol="Vectorized", numFeatures=200)


#réduire le poids des mots fréquents qui apparaissent dans de nombreux documents, 
#car ils peuvent ne pas être discriminants pour la classification. À l'inverse, l'IDF 
#accorde une plus grande importance aux mots rares qui apparaissent dans peu de documents, car ils sont plus discriminants.
idf = IDF(inputCol="Vectorized", outputCol="features")

rf = RandomForestClassifier(labelCol="toxic", featuresCol="features")

pipeline = Pipeline(stages=[hashingTF, idf, rf])   # va permettre d'exécuter plusieurs étapes en séquence pour transformer les données et entraîner le modèle de classification.

(trainingData, testData) = df_clean_text.randomSplit([0.7, 0.3], seed=42)

model = pipeline.fit(trainingData)

predictions = model.transform(testData)

evaluator = MulticlassClassificationEvaluator(labelCol="toxic", predictionCol="prediction", metricName="fMeasureByLabel")
accuracy = evaluator.evaluate(predictions)
print(" f1 = %g" % (accuracy))



 f1 = 0.892884


In [61]:
#le modèle prédit les commentaires toxic à 80%
tp = predictions[(predictions.toxic == 1) & (predictions.prediction == 1)].count()
fp = predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].count()

print("True positives = {}".format(tp)  )
print("False negatives = {}".format(fp))

True positives = 2
False negatives = 560


In [36]:
#randomn forest

#L'argument numFeatures dans la fonction HashingTF spécifie 
#le nombre de fonction de hachage à utiliser pour la transformation des mots en vecteurs
hashingTF = HashingTF(inputCol="text_cleaned", outputCol="Vectorized", numFeatures=200)


#réduire le poids des mots fréquents qui apparaissent dans de nombreux documents, 
#car ils peuvent ne pas être discriminants pour la classification. À l'inverse, l'IDF 
#accorde une plus grande importance aux mots rares qui apparaissent dans peu de documents, car ils sont plus discriminants.
idf = IDF(inputCol="Vectorized", outputCol="features")

rf = RandomForestClassifier(labelCol="toxic", featuresCol="features",weightCol="poid")

pipeline = Pipeline(stages=[hashingTF, idf, rf])   # va permettre d'exécuter plusieurs étapes en séquence pour transformer les données et entraîner le modèle de classification.

(trainingData, testData) = df_clean_text_fuck.randomSplit([0.7, 0.3], seed=42)

model = pipeline.fit(trainingData)

predictions = model.transform(testData)

evaluator = MulticlassClassificationEvaluator(labelCol="toxic", predictionCol="prediction", metricName="fMeasureByLabel")
accuracy = evaluator.evaluate(predictions)




In [37]:
print("fmeasurebylabel = %g" % auc)
tp = predictions[(predictions.toxic == 1) & (predictions.prediction == 1)].count()
fn = predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].count()
fp=  predictions[(predictions.toxic == 0) & (predictions.prediction == 1)].count()
print("True positives = {}".format(tp)  )
print("False negatives = {}".format(fn))
print("False positives = {}".format(fp))

fmeasurebylabel = 0.921969
True positives = 562
False negatives = 0
False positives = 2334


In [41]:
#randomn forest metric auc


#L'argume
#nt numFeatures dans la fonction HashingTF spécifie 
#le nombre de fonction de hachage à utiliser pour la transformation des mots en vecteurs
hashingTF = HashingTF(inputCol="text_cleaned", outputCol="Vectorized", numFeatures=200)


#réduire le poids des mots fréquents qui apparaissent dans de nombreux documents, 
#car ils peuvent ne pas être discriminants pour la classification. À l'inverse, l'IDF 
#accorde une plus grande importance aux mots rares qui apparaissent dans peu de documents, car ils sont plus discriminants.
idf = IDF(inputCol="Vectorized", outputCol="features")

dt = DecisionTreeClassifier(featuresCol="features", labelCol="toxic")


pipeline = Pipeline(stages=[hashingTF, idf, dt])   # va permettre d'exécuter plusieurs étapes en séquence pour transformer les données et entraîner le modèle de classification.

(trainingData, testData) = df_clean_text.randomSplit([0.7, 0.3], seed=42)

paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .build()

evaluator = MulticlassClassificationEvaluator(labelCol="toxic", predictionCol="prediction", metricName="fMeasureByLabel")

cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=3)

cvModel = cv.fit(trainingData)

predictions = cvModel.transform(testData)

accuracy = evaluator.evaluate(predictions)
print("fmeasurebylabel = %g" % accuracy)

bestParams = cvModel.bestModel.stages[-1].extractParamMap()

print("Les meilleurs paramètres sont : \n {}".format(bestParams))






fmeasurebylabel = 0.892165
Les meilleurs paramètres sont : 
 {Param(parent='DecisionTreeClassifier_a78c48b2a9a2', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False, Param(parent='DecisionTreeClassifier_a78c48b2a9a2', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10, Param(parent='DecisionTreeClassifier_a78c48b2a9a2', name='featuresCol', doc='features column name.'): 'features', Param(parent='DecisionTreeClassifier_a78c48b2a9a2', name='impurity', doc='Criterion used for information gain calculation (c

In [42]:
tp = predictions[(predictions.toxic == 1) & (predictions.prediction == 1)].count()
fp = predictions[(predictions.toxic == 0) & (predictions.prediction == 1)].count()
fn=  predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].count()

print("True positives = {}".format(tp)  )
print("False positives = {}".format(fp))
print("False negatives = {}".format(fn))


True positives = 153
False positives = 125
False negatives = 409


In [46]:
#randomn forest metric auc


#L'argume
#nt numFeatures dans la fonction HashingTF spécifie 
#le nombre de fonction de hachage à utiliser pour la transformation des mots en vecteurs
hashingTF = HashingTF(inputCol="text_cleaned", outputCol="Vectorized", numFeatures=200)


#réduire le poids des mots fréquents qui apparaissent dans de nombreux documents, 
#car ils peuvent ne pas être discriminants pour la classification. À l'inverse, l'IDF 
#accorde une plus grande importance aux mots rares qui apparaissent dans peu de documents, car ils sont plus discriminants.
idf = IDF(inputCol="Vectorized", outputCol="features")

dt = DecisionTreeClassifier(featuresCol="features", labelCol="toxic",weightCol="poid")


pipeline = Pipeline(stages=[hashingTF, idf, dt])   # va permettre d'exécuter plusieurs étapes en séquence pour transformer les données et entraîner le modèle de classification.

(trainingData, testData) = df_clean_text_fuck.randomSplit([0.7, 0.3], seed=42)

paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .build()

evaluator = MulticlassClassificationEvaluator(labelCol="toxic", predictionCol="prediction", metricName="fMeasureByLabel")

cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=3)

cvModel = cv.fit(trainingData)

predictions = cvModel.transform(testData)

accuracy = evaluator.evaluate(predictions)
print("fmeasurebylabel = %g" % accuracy)

bestParams = cvModel.bestModel.stages[-1].extractParamMap()

print("Les meilleurs paramètres sont : \n {}".format(bestParams))






fmeasurebylabel = 0
Les meilleurs paramètres sont : 
 {Param(parent='DecisionTreeClassifier_377bdb4077d5', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False, Param(parent='DecisionTreeClassifier_377bdb4077d5', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10, Param(parent='DecisionTreeClassifier_377bdb4077d5', name='featuresCol', doc='features column name.'): 'features', Param(parent='DecisionTreeClassifier_377bdb4077d5', name='impurity', doc='Criterion used for information gain calculation (case-ins

In [47]:
tp = predictions[(predictions.toxic == 1) & (predictions.prediction == 1)].count()
fp = predictions[(predictions.toxic == 0) & (predictions.prediction == 1)].count()
fn=  predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].count()

print("True positives = {}".format(tp)  )
print("False positives = {}".format(fp))
print("False negatives = {}".format(fn))

True positives = 562
False positives = 2334
False negatives = 0


In [33]:


hashingTF = HashingTF(inputCol="text_cleaned", outputCol="Vectorized", numFeatures=200)

#réduire le poids des mots fréquents qui apparaissent dans de nombreux documents, 
#car ils peuvent ne pas être discriminants pour la classification. À l'inverse, l'IDF 
#accorde une plus grande importance aux mots rares qui apparaissent dans peu de documents, car ils sont plus discriminants.
idf = IDF(inputCol="Vectorized", outputCol="features")

rf = RandomForestClassifier(labelCol="toxic", featuresCol="features")

pipeline = Pipeline(stages=[hashingTF, idf, rf])   # va permettre d'exécuter plusieurs étapes en séquence pour transformer les données et entraîner le modèle de classification.

(trainingData, testData) = df_clean_text.randomSplit([0.7, 0.3], seed=42)

paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .build()

evaluator = MulticlassClassificationEvaluator(labelCol='toxic', predictionCol='prediction', metricName='fMeasureByLabel')

cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=3)

cvModel = cv.fit(trainingData)

predictions = cvModel.transform(testData)

accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)

bestParams = cvModel.bestModel.stages[-1].extractParamMap()

print("Les meilleurs paramètres sont : \n {}".format(bestParams))







Accuracy = 0.902297
Les meilleurs paramètres sont : 
 {Param(parent='RandomForestClassifier_07b70563dcb8', name='bootstrap', doc='Whether bootstrap samples are used when building trees.'): True, Param(parent='RandomForestClassifier_07b70563dcb8', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False, Param(parent='RandomForestClassifier_07b70563dcb8', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10, Param(parent='RandomForestClassifier_07b70563dcb8', name='featureSubsetStrategy', doc="The number of feat

In [28]:

tp = predictions[(predictions.toxic == 1) & (predictions.prediction == 1)].count()
fp = predictions[(predictions.toxic == 0) & (predictions.prediction == 1)].count()
fn=  predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].count()

print("True positives = {}".format(tp)  )
print("False positives = {}".format(fp))
print("False negatives = {}".format(fn))


True positives = 76
False positives = 16
False negatives = 486


In [31]:
#randomn forest metric auc


#L'argume
#nt numFeatures dans la fonction HashingTF spécifie 
#le nombre de fonction de hachage à utiliser pour la transformation des mots en vecteurs
hashingTF = HashingTF(inputCol="text_cleaned", outputCol="Vectorized", numFeatures=200)


#réduire le poids des mots fréquents qui apparaissent dans de nombreux documents, 
#car ils peuvent ne pas être discriminants pour la classification. À l'inverse, l'IDF 
#accorde une plus grande importance aux mots rares qui apparaissent dans peu de documents, car ils sont plus discriminants.
idf = IDF(inputCol="Vectorized", outputCol="features")

rf = RandomForestClassifier(labelCol="toxic", featuresCol="features")

pipeline = Pipeline(stages=[hashingTF, idf, rf])   # va permettre d'exécuter plusieurs étapes en séquence pour transformer les données et entraîner le modèle de classification.

(trainingData, testData) = df_clean_text.randomSplit([0.7, 0.3], seed=42)

paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .build()

evaluator =  MulticlassClassificationEvaluator(labelCol='toxic', predictionCol='prediction', metricName='fMeasureByLabel')

cv = CrossValidator(estimator=pipeline, evaluator=evaluator, estimatorParamMaps=paramGrid, numFolds=3)

cvModel = cv.fit(trainingData)

predictions = cvModel.transform(testData)

accuracy = evaluator.evaluate(predictions)
print("f1 = %g" % accuracy)



f1 = 0.902297


In [32]:
bestParams = cvModel.bestModel.stages[-1].extractParamMap()

#print("Les meilleurs paramètres sont : \n {}".format(bestParams))


tp = predictions[(predictions.toxic == 1) & (predictions.prediction == 1)].count()
fp = predictions[(predictions.toxic == 0) & (predictions.prediction == 1)].count()
fn=  predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].count()

print("True positives = {}".format(tp)  )
print("False positives = {}".format(fp))
print("False negatives = {}".format(fn))

#find synonym dans word2vec pour voir si les mots prédits comme toxiques sont bon
#utiliser un maxiter supérieur à 2
#f1 faiqs la moyenne sur toutes les classes
#metricLabel=1
#metricname fMeasureLabel

True positives = 76
False positives = 16
False negatives = 486


In [84]:
df_clean_text_fuck.show(truncate=False)

+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# W2VEC

In [83]:
w2v = Word2Vec(vectorSize=300, seed=42, inputCol="text_cleaned", outputCol="Vectorized")
rf = RandomForestClassifier(featuresCol="Vectorized", labelCol="toxic", weightCol = "poid")
w2v_model = w2v.fit(df_clean_text_fuck)
w2v_transformed = w2v_model.transform(df_clean_text_fuck)


param_grid = ParamGridBuilder() \
             .addGrid(rf.numTrees, [10, 15, 20]) \
             .addGrid(rf.maxDepth, [5, 10]) \
             .build()


# Split the data into training and test sets
train_data, test_data = w2v_transformed.randomSplit([0.7, 0.3])


evaluator =  MulticlassClassificationEvaluator(labelCol='toxic', predictionCol='prediction', metricName='fMeasureByLabel')

cv = CrossValidator(estimator=rf,
                    estimatorParamMaps=param_grid,
                    evaluator=evaluator,
                    numFolds=3,
                    )

cvModel = cv.fit(trainingData)

predictions = cvModel.transform(testData)

fmeasure = evaluator.evaluate(predictions)

print("fmeasure randomn forest :", fmeasure)


fmeasure randomn forest : 0.0
True positives = 562
False negatives = 0


In [ ]:
tp = predictions[(predictions.toxic == 1) & (predictions.prediction == 1)].count()
fp = predictions[(predictions.toxic == 0) & (predictions.prediction == 1)].count()
fn=  predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].count()

print("True positives = {}".format(tp)  )
print("False positives = {}".format(fp))
print("False negatives = {}".format(fn))

In [29]:
# Transform the input text data into vector representations
w2v = Word2Vec(vectorSize=300, seed=42, inputCol="text_cleaned", outputCol="Vectorized", maxIter = 15)
w2v_model = w2v.fit(df_clean_text_fuck)
w2v_transformed = w2v_model.transform(df_clean_text_fuck)
rf = RandomForestClassifier(featuresCol="Vectorized", labelCol="toxic", weightCol = "poid")

# Split the data into training and test sets
train_data, test_data = w2v_transformed.randomSplit([0.7, 0.3])

evaluator =  MulticlassClassificationEvaluator(labelCol='toxic', predictionCol='prediction',metricLabel = 1, metricName='fMeasureByLabel')

lr_model = rf.fit(train_data)

lr_accuracy = evaluator.evaluate(lr_model.transform(test_data))
print("f measure label:", lr_accuracy)

f measure label: 0.3302598491198659


In [30]:
tp = predictions[(predictions.toxic == 1) & (predictions.prediction == 1)].count()
fp = predictions[(predictions.toxic == 0) & (predictions.prediction == 1)].count()
fn=  predictions[(predictions.toxic == 1) & (predictions.prediction == 0)].count()

print("True positives = {}".format(tp)  )
print("False positives = {}".format(fp))
print("False negatives = {}".format(fn))

True positives = 76
False positives = 16
False negatives = 486
